In [1]:
import pandas as pd 

In [26]:
df_all = pd.read_csv('comment_list.csv')
df_all

,segment_id,start_time_sec,end_time_sec,transcription,topic_tag,reading,filename
0,8,32.946179,39.994809,でデイジーはですねこれマリオランドゲームボーイのマリオランドに出てくるねピーチ姫的なね立ち位...,ファイター,ででいじーわですねこれまりおらんどげーむぼーいのまりおらんどにでてくるねぴーちひめてきなねた...,FF1output_id8_delay-1.5.mp4
1,8,32.946179,39.994809,でデイジーはですねこれマリオランドゲームボーイのマリオランドに出てくるねピーチ姫的なね立ち位...,ファイター,ででいじーわですねこれまりおらんどげーむぼーいのまりおらんどにでてくるねぴーちひめてきなねた...,FF1output_id8_delay-1.25.mp4
2,8,32.946179,39.994809,でデイジーはですねこれマリオランドゲームボーイのマリオランドに出てくるねピーチ姫的なね立ち位...,ファイター,ででいじーわですねこれまりおらんどげーむぼーいのまりおらんどにでてくるねぴーちひめてきなねた...,FF1output_id8_delay-1.0.mp4
3,8,32.946179,39.994809,でデイジーはですねこれマリオランドゲームボーイのマリオランドに出てくるねピーチ姫的なね立ち位...,ファイター,ででいじーわですねこれまりおらんどげーむぼーいのまりおらんどにでてくるねぴーちひめてきなねた...,FF1output_id8_delay-0.75.mp4
4,8,32.946179,39.994809,でデイジーはですねこれマリオランドゲームボーイのマリオランドに出てくるねピーチ姫的なね立ち位...,ファイター,ででいじーわですねこれまりおらんどげーむぼーいのまりおらんどにでてくるねぴーちひめてきなねた...,FF1output_id8_delay-0.5.mp4
...,...,...,...,...,...,...,...
25075,30,162.502642,165.758147,これはもうサムス素晴らしかったと思います,対戦風景,これわもーさむすすばらしかったとおもいます,MM4output_id30_delay0.75.mp4
25076,30,162.502642,165.758147,これはもうサムス素晴らしかったと思います,対戦風景,これわもーさむすすばらしかったとおもいます,MM4output_id30_delay1.0.mp4
25077,30,162.502642,165.758147,これはもうサムス素晴らしかったと思います,対戦風景,これわもーさむすすばらしかったとおもいます,MM4output_id30_delay1.25.mp4
25078,30,162.502642,165.758147,これはもうサムス素晴らしかったと思います,対戦風景,これわもーさむすすばらしかったとおもいます,MM4output_id30_delay1.5.mp4


In [21]:
def set_sample(df):
    
    # CSVファイルを読み込み、元の行番号を保持
    #df = pd.read_csv('./comment_list2.csv')  # 実際のファイル名に変更してください
    #df.reset_index(inplace=True)  # 元の行番号を保持
    # df.rename(columns={'index': 'Row'}, inplace=True)  # 'Row'という列名に変更
    df["Row"] = df.index

    # 'filename'列から'delay'の後の数値を抽出し、新しい列 'delay_value' として追加
    df['delay_value'] = df['filename'].str.extract(r'delay([\-]?\d+\.\d+)')[0].astype(float)

    # 指定された12種類のdelay値
    unique_delays = [-1.5, -1.25, -1.0, -0.75, -0.5, 0.0, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75]
    result = []
    set_num = 1  # セット番号の初期化

    # 各 delay_value を2回ずつ含む24個のセットを作成し、データが足りる限り繰り返す
    while True:
        set_group = []

        # 各 delay_value に対して2つのファイルをランダムに選びセットを作成
        for delay in unique_delays:
            delay_group = df[df['delay_value'] == delay]
            
            # 各 delay に対して2つのファイルをランダムに選ぶ
            if len(delay_group) >= 2:
                selected_rows = delay_group.sample(2)
                set_group.extend(selected_rows[['Row', 'filename', 'topic_tag']].values.tolist())
            else:
                # ファイルが足りなくなった場合、セットを破棄して再試行
                set_group = []
                break

        # 重複のチェック
        rows = [row[0] for row in set_group]
        if len(rows) != len(set(rows)):
            # 重複があればセットの作成をやり直す
            continue

        # セットが24個に満たない場合もループを終了
        if len(set_group) < 24:
            break

        # 重複もなく、24個のセットが完成した場合のみ、選択したファイルを元の DataFrame から削除
        for row in rows:
            result.append([row, set_group[rows.index(row)][1], set_group[rows.index(row)][2], f'Set(2){set_num}'])
            df = df[df['Row'] != row]
        
        # セット番号をインクリメント
        set_num += 1
    
    return result


In [ ]:
# def set_sample(df):
#     # 'filename'列から'delay'の後の数値を抽出し、新しい列 'delay_value' として追加
#     df['delay_value'] = df['filename'].str.extract(r'delay([\-]?\d+\.\d+)')[0].astype(float)
#     df["set_id"] = None

#     # 指定された12種類のdelay値
#     unique_delays = [-1.5, -1.25, -1.0, -0.75, -0.5, 0.0, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75]
#     result = []
#     set_num = 1  # セット番号の初期化


#     # 各 delay_value を2回ずつ含む24個のセットを作成し、データが足りる限り繰り返す
#     while True:
#         set_group = []

#         # 各 delay_value に対して2つのファイルをランダムに選びセットを作成
#         for delay in unique_delays:
#             delay_group = df[df['delay_value'] == delay]
            
#             # 各 delay に対して2つのファイルをランダムに選ぶ
#             if len(delay_group) >= 2:
#                 selected_rows = delay_group.sample(2)
#                 set_group.extend(selected_rows[['filename', 'topic_tag']].values.tolist())
#             else:
#                 # ファイルが足りなくなった場合、セットを破棄して再試行
#                 set_group = []
#                 break

#         # 重複のチェック
#         rows = [row[0] for row in set_group]
#         if len(rows) != len(set(rows)):
#             # 重複があればセットの作成をやり直す
#             continue

#         # セットが24個に満たない場合もループを終了
#         if len(set_group) < 24:
#             break

#         # 重複もなく、24個のセットが完成した場合のみ、選択したファイルを元の DataFrame から削除
#         #for row in rows:
#         #    result.append([row, set_group[rows.index(row)][1], set_group[rows.index(row)][2], f'Set(2){set_num}'])
#         #    df = df[df['Row'] != row]
        
#         # セット番号をインクリメント
#         set_num += 1
    
#     return df


In [31]:
from kl_set import calculate_kl_divergence_sum, count_topic_tag_proportions

# 全体の統計量
reference_proportions = count_topic_tag_proportions(df_all)
df_sub = df_all.copy()

while True:
    # set id の割り振り
    result = set_sample(df_sub.copy())
    df_loop = pd.DataFrame(result, columns=['Row', 'filename', 'topic_tag', 'set_id'])

    # score の計算
    scores = {}
    for set_value, group_df in df_loop.groupby('set_id'):
        partial_propotions = count_topic_tag_proportions(group_df)
        kl_divergence = calculate_kl_divergence_sum(reference_proportions, partial_propotions)

        scores[set_value] = kl_divergence
    
    # 該当しなかったものの排除
    eliminated_set = [k for k, v in scores.items() if v > 0.1] # KL div が 0.1 を超えた set_id が入っている
    df_sub = df_loop[df_loop['set_id'].isin(eliminated_set)]


,Row,filename,topic_tag,set_id
0,21732,FF3output_id21_delay-1.5.mp4,対戦風景,Set(2)1
1,7428,MF11output_id10_delay-1.5.mp4,ファイター,Set(2)1
2,14041,MF13output_id31_delay-1.25.mp4,対戦風景,Set(2)1
3,14689,MF21output_id10_delay-1.25.mp4,対戦風景,Set(2)1
4,18986,MM1output_id1_delay-1.0.mp4,ファイター,Set(2)1
...,...,...,...,...
25075,11169,FF2output_id10_delay1.25.mp4,アイテム,Set(2)1045
25076,3514,MM3output_id27_delay1.5.mp4,対戦風景,Set(2)1045
25077,23722,MM4output_id8_delay1.5.mp4,アイテム,Set(2)1045
25078,25055,MM4output_id30_delay1.75.mp4,対戦風景,Set(2)1045
